## CycleGAN

* task: image translation: map from zebra/horse space to horse/zebra space
* model: $G$
* loss term: GAN loss + cycle loss + identity loss

In [ ]:
import torch
import torch.nn as nn
import torchvision

import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

import torchvision.transforms as transforms
from PIL import Image

from tqdm import tqdm
import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## define model

In [ ]:
class Gen(nn.Module):
    def __init__(self) -> None:
        super(Gen, self).__init__()
    def forward(self, x):

        return x

In [ ]:
class Disc(nn.Module):
    """
    input: (N, 3, 256, 256)
    output: (N, 1, 30, 30)
    """

    def __init__(self, img_channel) -> None:
        super(Disc, self).__init__()



    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = self.sigmoid(x)
        return x

## define some params

In [ ]:
LEARNING_RATE_GEN = 2e-4
LEARNING_RATE_DISC = 2e-4
LAMBDA_CYCLE = 10
LAMBDA_INDENTITY = 5
IMAGE_CHANNEL = 3
NUM_EPOCH = 10
BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## prepare data

In [ ]:
# download and unzip dataset
! wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip && unzip horse2zebra.zip

In [ ]:
class horse_zebra(Dataset):
    def __init__(self, transform, path_horse="./horse2zebra/trainA/",path_zebra="./horse2zebra/trainB/"):
        self.path_horse = path_horse
        self.path_zebra = path_zebra

        self.transform = transform

        self.lst_horse = os.listdir(self.path_horse)
        self.lst_zebra = os.listdir(self.path_zebra)

        # one thing about the dataset:
        # the num of horse pic != num of zebra pic
        self.length = max(len(self.lst_horse), len(self.lst_zebra))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # some pic on the dataset just have one channel
        # so need to use function: convert("RGB")
        horse = Image.open(self.path_horse + self.lst_horse[torch.randint(len(self.lst_horse), size=(1,))]).convert("RGB")
        zebra = Image.open(self.path_zebra + self.lst_zebra[idx]).convert("RGB")
        if self.transform:
            horse = self.transform(horse)
            zebra = self.transform(zebra)
        return horse, zebra



In [ ]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
])

In [ ]:
dataset = horse_zebra(transform)
dataloader = DataLoader(dataset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
# get test samples
# here, i choose to pick images in training set to visiualize whether the generator is learning
fix_horse, fix_zebra = next(iter(dataloader))

## keep the image buffer

in the paper, they do not put the image newly generated by Gen into Disc, instead, they keep a buffer of generated image and then sample image from the buffer to Disc.

In [ ]:
class ImageBuffer:
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []

    def get_image(self, data):
        # data is what Gen generated and send in func
        # we will return a tensor of the same shape as data send in
        tensor_return = []
        for each in data:
            each = each.unsqueeze(0)
            if len(self.data) < self.max_size:
                self.data.append(each)
                tensor_return.append(each)
            else:
                # with prob == 0.5
                # we will update the buffer
                if torch.randn(1) < 0.5:
                    choose = torch.randint(self.max_size, size=(1,))
                    tensor_return.append(self.data[choose])
                    self.data[choose] = each
                else:
                    tensor_return.append(each)
        return torch.cat(tensor_return, dim=0)


## define optim and loss

In [ ]:
def weight_init(m):
    """
    init model's prams, if a layer is a conv layer: set the weight to N(0, 0.02), set the bias to constant 0.
    if a layer is a batch norm: set the weight to N(1, 0.02), set the bias to constant 0.
    usage: model.apply(weight_init)

    params:
        m: model's layer
    return:
        None
    """
    class_name = m.__class__.__name__
    if class_name.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0)
    if class_name.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1, 0.02)
        torch.nn.init.constant_(m.bias.data, 0)


In [ ]:
G_H_Z = Gen(IMAGE_CHANNEL).to(DEVICE)
G_Z_H = Gen(IMAGE_CHANNEL).to(DEVICE)

D_H = Disc(IMAGE_CHANNEL).to(DEVICE)
D_Z = Disc(IMAGE_CHANNEL).to(DEVICE)

G_H_Z.apply(weight_init)
G_Z_H.apply(weight_init)
D_H.apply(weight_init)
D_Z.apply(weight_init)

# optim two networks' params at the same time
opt_gen = optim.Adam(list(G_Z_H.parameters()) + list(G_H_Z.parameters()), lr=LEARNING_RATE_GEN, betas=(0.5, 0.999))
opt_d_h = optim.Adam(D_H.parameters(), lr=LEARNING_RATE_DISC, betas=(0.5, 0.999))
opt_d_z = optim.Adam(D_Z.parameters(), lr=LEARNING_RATE_DISC, betas=(0.5, 0.999))

gan_criterion = nn.MSELoss()
cycle_criterion = nn.L1Loss()
identity_criterion = nn.L1Loss()

model_checkpoint = {
    'G_H_Z':None,
    'G_Z_H':None,
    'D_Z':None,
    'D_H':None
}


## set up the training loop

In [ ]:
! mkdir ./horse_output && mkdir ./zebra_output

In [ ]:
# function to sample images
def sample_image(fix_horse, fix_zebra):
    fake_zebra = G_H_Z(fix_horse)
    fake_horse = G_Z_H(fix_zebra)
    cycle_horse = G_Z_H(fake_zebra)
    cycle_zebra = G_H_Z(fake_horse)

    grid_real_horse = torchvision.utils.make_grid(fix_horse, nrow=1, normalize=True)
    grid_fake_zebra = torchvision.utils.make_grid(fake_zebra, nrow=1, normalize=True)
    grid_cycle_horse = torchvision.utils.make_grid(cycle_horse, nrow=1, normalize=True)
    grid_horse = torch.cat((grid_real_horse, grid_fake_zebra, grid_cycle_horse), dim=2)

    grid_real_zebra = torchvision.utils.make_grid(fix_zebra, nrow=1, normalize=True)
    grid_fake_horse = torchvision.utils.make_grid(fake_horse, nrow=1, normalize=True)
    grid_cycle_zebra = torchvision.utils.make_grid(cycle_zebra, nrow=1, normalize=True)
    grid_zebra = torch.cat((grid_real_zebra, grid_fake_horse, grid_cycle_zebra), dim=2)
    torchvision.utils.save_image(grid_horse, f"./horse_output/horse_epoch{epoch}_batch_{batch_idx}.png", normalize=False)
    torchvision.utils.save_image(grid_zebra, f"./zebra_output/zebra_epoch{epoch}_batch_{batch_idx}.png", normalize=False)
    print("saving samples... done!")
def save_checkpoint(dict, path="model.pth.tar"):
    """
    save the model and optim through a dictionary
     {"model": model.state_dict(), "optim": optim.state_dict()} (*)


    params:
        data: a dict of the structure (*) storing the model and optim

    return:
        None
    """
    now = time.strftime("%D_%H:%M")
    print(f"saving checkpoint at {now}, path is '{path}'")
    torch.save(dict, path)
    print("saving model... done!")

In [ ]:
fake_zebra_buffer = ImageBuffer()
fake_horse_buffer = ImageBuffer()

In [ ]:
for epoch in range(NUM_EPOCH):
    loop = tqdm(dataloader, leave=True)
    for batch_idx, (horse, zebra) in enumerate(loop):
        G_H_Z.train()
        G_Z_H.train()

        horse = horse.to(DEVICE)
        zebra = zebra.to(DEVICE)

        #### train Gen ####
        opt_gen.zero_grad()

        # train generator with the lastest generated image
        fake_horse = G_Z_H(zebra)
        d_h_fake = D_H(fake_horse)
        fake_zebra = G_H_Z(horse)
        d_z_fake = D_Z(fake_zebra)
        # gan loss
        gan_loss = gan_criterion(d_z_fake, torch.ones_like((d_z_fake)).to(DEVICE)) + gan_criterion(d_h_fake, torch.ones_like(d_h_fake))

        # identity loss
        identity_zebra = G_H_Z(zebra)
        identity_horse = G_Z_H(horse)
        identity_loss = (identity_criterion(identity_zebra, zebra) + identity_criterion(identity_horse, horse)) * LAMBDA_INDENTITY

        # cycle loss
        cycle_horse = G_Z_H(fake_zebra)
        cycle_zebra = G_H_Z(fake_horse)
        cycle_loss = (cycle_criterion(cycle_horse, horse) + cycle_criterion(cycle_zebra, zebra)) * LAMBDA_CYCLE

        Gen_loss = gan_loss + identity_loss + cycle_loss
        Gen_loss.backward()
        opt_gen.step()

        #### train discriminator ####
        opt_d_h.zero_grad()
        # train discriminator with the image from buffer

        d_h_real = D_H(horse)
        # get image from buffer
        fake_horse_ = fake_horse_buffer.get_image(fake_horse)
        d_h_fake = D_H(fake_horse_.detach())
        d_h_loss = gan_criterion(d_h_real, torch.ones_like(d_h_real).to(DEVICE)) + \
            gan_criterion(d_h_fake, torch.zeros_like(d_h_fake).to(DEVICE))
        d_h_loss.backward()
        opt_d_h.step()

        opt_d_z.zero_grad()
        d_z_real = D_Z(zebra)
        fake_zebra_ = fake_zebra_buffer.get_image(fake_zebra)
        d_z_fake = D_Z(fake_zebra_.detach())
        d_z_loss = gan_criterion(d_z_real, torch.ones_like((d_z_real).to(DEVICE))) + \
            gan_criterion(d_z_fake, torch.zeros_like(d_z_fake).to(DEVICE))
        d_z_loss.backward(retain_graph=True)
        opt_d_z.step()

        if batch_idx % 40 == 0:
          G_Z_H.eval()
          G_H_Z.eval()
          with torch.no_grad():
                # sample gans output
                sample_image(fix_horse.to(DEVICE), fix_zebra.to(DEVICE))

    # saving models
    model_checkpoint['G_H_Z']=G_H_Z.state_dict()
    model_checkpoint['G_Z_H']=G_Z_H.state_dict()
    model_checkpoint['D_Z']=D_Z.state_dict()
    model_checkpoint['D_H']=D_H.state_dict()
    save_checkpoint(model_checkpoint)

In [ ]:
!ls ./horse_output

In [ ]:
! zip -r zebraoutput.zip zebra_output